# Linked Data and Ontologies for Construction Management


We will be using [Csite ontology](https://www.sciencedirect.com/science/article/pii/S0926580523004843) to demonstrate how linked data helps construction management. You can cite csite ontology as below if you need to use it in your work.

>Farghaly, Karim, Ranjith Soman, and Jennifer Whyte. "cSite ontology for production control of construction sites." Automation in Construction 158 (2024): 105224.

# Import necessary libraries


**pandas** is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool, built on top of the Python programming language.pandas can be installed via pip from PyPI.

`pip install pandas`

RDFLib is open source and is maintained in a GitHub repository. RDFLib releases, current and previous, are listed on PyPi

The best way to install RDFLib is to use pip (sudo as required):

`pip install rdflib`
If you want the latest code to run, clone the main branch of the GitHub repo and use that or you can pip install directly from GitHub:

`pip install git+https://github.com/RDFLib/rdflib.git@main#egg=rdflib`

In [1]:
import rdflib
import owlrl
import pandas as pd

# Define functions to load ttl file and perform SPARQL query

Define the function to load a Turtle file into an RDF graph

In [2]:
def load_ttl_file(file_path):
    graph = rdflib.Graph()
    graph.parse(file_path, format="ttl")
    return graph

Define the function to perform a SPARQL query

In [3]:
def perform_sparql_query(graph, query):
    """ Perform a SPARQL query on the RDFLib graph. """
    return graph.query(query)

def display_query_results(results):
    """ Display SPARQL query results as a pandas DataFrame. """
    # Transform the results into a list of dictionaries
    uri_to_remove = "http://www.owl-ontologies.com/"
    
    # Transform the results into a list of dictionaries, removing the specified URI part
    data = []
    for row in results:
        row_dict = {}
        for field in results.vars:
            value = str(row[field])
            # Remove the unwanted URI part if it's present
            if value.startswith(uri_to_remove):
                value = value[len(uri_to_remove):]
            row_dict[str(field)] = value
        data.append(row_dict)
    
    # Create and display a DataFrame
    return pd.DataFrame(data)

## Load the turtle file

In [4]:
ttl_file = "ProjectGraph.ttl"  # Change this to your Turtle file path
graph = load_ttl_file(ttl_file)


# Define SPARQL query and return the outputs

## Query 1 :Identifying deliverables per floor

The first query is about tracking the number of deliverables (activities to be delivered)per floor in a weekplan. This helps the project managers to understand the plans and remove workspace conflicts. It also helps the clustering of work to be distributed to different package managers.  For this, the query first looks at each week’s plan and then identify the activities and their location and then groups it by storey.

In [5]:
# Define a SPARQL query



query = """
PREFIX cSite: <http://www.owl-ontologies.com/cSite#>
PREFIX bot: <http://www.owl-ontologies.com/bot#>



SELECT ?storey ?activity ?activityversion
WHERE {
  ?activity cSite:hasZone ?space.
  ?space bot:hasStorey ?storey.
  ?activity cSite:PlannedVersion ?activityversion.
  ?activity cSite:discussedIn cSite:THISWEEK-WeeklyPMUpdate220401.xlsx.
}
GROUP BY ?storey ?activity
"""


results = perform_sparql_query(graph, query)

# Display results
df = display_query_results(results)
print(df)

       storey                 activity activityversion
0   cSite#001  cSite#3450-QSO-001-0001               1
1   cSite#002  cSite#3450-QSO-002-0001               1
2   cSite#006  cSite#3800-QSO-006-0007               1
3   cSite#006  cSite#3800-QSO-006-PS01               1
4   cSite#006  cSite#3800-QSO-006-PSGF               1
5   cSite#005  cSite#4255-QHP-005-MB28               1
6   cSite#001  cSite#4255-QSO-001-B228               1
7   cSite#002  cSite#4255-QSO-002-0020               1
8   cSite#003  cSite#4255-QSO-003-0022               1
9   cSite#003  cSite#4255-QSO-003-0024               1
10  cSite#001  cSite#4300-QSO-001-1401               1
11  cSite#001  cSite#4446-QSO-001-0076               1
12  cSite#001  cSite#4446-QSO-001-0077               1
13  cSite#001  cSite#4446-QSO-001-0078               1
14  cSite#001  cSite#4446-QSO-001-0079               1
15  cSite#001  cSite#4446-QSO-001-0080               1
16  cSite#002  cSite#4446-QSO-002-0473               1
17  cSite#



### Prefixes
The prefixes are shorthand notations used to simplify the query:
- `cSite: <http://www.owl-ontologies.com/cSite#>`
- `bot: <http://www.owl-ontologies.com/bot#>`

These define namespaces for the properties and classes that will be used in the query.

### SELECT Clause
```sparql
SELECT ?storey ?activity ?activityversion
```
This part of the query specifies that the results will include three variables:
- `?storey`
- `?activity`
- `?activityversion`

### WHERE Clause
```sparql
WHERE {
  ?activity cSite:hasZone ?space.
  ?space bot:hasStorey ?storey.
  ?activity cSite:PlannedVersion ?activityversion.
  ?activity cSite:discussedIn cSite:THISWEEK-WeeklyPMUpdate220401.xlsx.
}
```
This part defines the pattern to match in the dataset. Let's break it down:
1. `?activity cSite:hasZone ?space.`:
   - This states that there is an `?activity` which is associated with a `?space` through the property `cSite:hasZone`.
   
2. `?space bot:hasStorey ?storey.`:
   - This states that the `?space` is associated with a `?storey` through the property `bot:hasStorey`.

3. `?activity cSite:PlannedVersion ?activityversion.`:
   - This states that the `?activity` has a planned version represented by the variable `?activityversion` through the property `cSite:PlannedVersion`.

4. `?activity cSite:discussedIn cSite:THISWEEK-WeeklyPMUpdate220401.xlsx.`:
   - This states that the `?activity` is discussed in the document `cSite:THISWEEK-WeeklyPMUpdate220401.xlsx`.

### GROUP BY Clause
```sparql
GROUP BY ?storey ?activity
```
This ensures that the results are grouped by `?storey` and `?activity`. It means the query will return unique combinations of `?storey` and `?activity`.

### Summary
In plain English, this SPARQL query retrieves information about activities discussed in a specific weekly update document. For each activity, it gets the storey (floor level) it is associated with, the activity itself, and its planned version. The results are grouped by the storey and the activity to ensure each combination is unique.

So, the query will return a list of activities, the storeys they are associated with, and their planned versions, but only if they are discussed in the specified weekly update document.

## Query 2: Tracking the deliverables per floor

The second query is about tracking the number of deliverables that has been completed in the last 6 weeks. This helps the project manager to understand the flow of work and plan the further activities. For this, the query first looks for all the activities in the last 6 weeks and infers where the activity occurred and then checks for the completion status. All the completed activities are then grouped by storey. 

In [6]:
# Define a SPARQL query



query = """
PREFIX cSite: <http://www.owl-ontologies.com/cSite#>
PREFIX bot: <http://www.owl-ontologies.com/bot#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?storey ?weekplan (COUNT(?activity) AS ?count) WHERE {
  ?weekplan cSite:CoversFrom ?date.
  ?activity cSite:discussedIn ?weekplan;
    cSite:hasZone ?space.
  ?space bot:hasStorey ?storey.
  ?activity cSite:StatusCompleted ?status.
  #BIND((NOW()) - "P0Y6M28DT0M0S"^^xsd:yearMonthDuration AS ?sixweeksago)
  BIND("2022-04-01"^^xsd:date AS ?sixweeksago)
  FILTER(?date > ?sixweeksago)
  FILTER(?status = "Yes"^^xsd:string)
}
GROUP BY ?storey ?weekplan
ORDER BY (?storey)

"""


results = perform_sparql_query(graph, query)

# Display results
df = display_query_results(results)
print(df)

       storey                                  weekplan count
0   cSite#001  cSite#THISWEEK-WeeklyPMUpdate220408.xlsx     1
1   cSite#001  cSite#THISWEEK-WeeklyPMUpdate220506.xlsx     6
2   cSite#001  cSite#THISWEEK-WeeklyPMUpdate220527.xlsx    30
3   cSite#001  cSite#THISWEEK-WeeklyPMUpdate220415.xlsx     6
4   cSite#002  cSite#THISWEEK-WeeklyPMUpdate220429.xlsx    10
5   cSite#002  cSite#THISWEEK-WeeklyPMUpdate220408.xlsx     7
6   cSite#002  cSite#THISWEEK-WeeklyPMUpdate220506.xlsx    14
7   cSite#002  cSite#THISWEEK-WeeklyPMUpdate220527.xlsx     4
8   cSite#002  cSite#THISWEEK-WeeklyPMUpdate220415.xlsx     9
9   cSite#002  cSite#THISWEEK-WeeklyPMUpdate220422.xlsx     4
10  cSite#003  cSite#THISWEEK-WeeklyPMUpdate220506.xlsx     2
11  cSite#020  cSite#THISWEEK-WeeklyPMUpdate220429.xlsx     1


This SPARQL query is designed to retrieve and count completed activities discussed in week plans that occurred after a specific date, grouped by storey and week plan. Here is a detailed explanation of each part of the query:

1. **Prefix Definitions:**
   - `PREFIX cSite: <http://www.owl-ontologies.com/cSite#>`: This defines a prefix `cSite` for the namespace related to construction site ontology.
   - `PREFIX bot: <http://www.owl-ontologies.com/bot#>`: This defines a prefix `bot` for the Building Topology Ontology namespace.
   - `PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>`: This defines a prefix `xsd` for XML Schema Definition, used for data types.

2. **SELECT Clause:**
   - `SELECT ?storey ?weekplan (COUNT(?activity) AS ?count)`: This specifies that the query will return three variables: the storey, the week plan, and the count of activities.

3. **WHERE Clause:**
   - `?weekplan cSite:CoversFrom ?date.`: This finds week plans that have a start date.
   - `?activity cSite:discussedIn ?weekplan;`: This finds activities discussed in those week plans.
     - `cSite:hasZone ?space.`: This specifies that each activity is associated with a space.
   - `?space bot:hasStorey ?storey.`: This finds the storey associated with that space.
   - `?activity cSite:StatusCompleted ?status.`: This finds activities that have a completion status.
   
   The BIND and FILTER operations are used to narrow down the results:
   - `BIND("2022-04-01"^^xsd:date AS ?sixweeksago)`: This binds a specific date ("2022-04-01") as the variable `?sixweeksago`.
   - `FILTER(?date > ?sixweeksago)`: This filters the week plans to include only those that started after the specified date.
   - `FILTER(?status = "Yes"^^xsd:string)`: This filters the activities to include only those that are marked as completed.

4. **GROUP BY Clause:**
   - `GROUP BY ?storey ?weekplan`: This groups the results by storey and week plan.

5. **ORDER BY Clause:**
   - `ORDER BY (?storey)`: This orders the results by storey.

**In Summary:**
The query retrieves the number of completed activities discussed in week plans starting after April 1, 2022, grouped by storey and week plan, and ordered by storey.

## Query 3: Identifying subcontractor productivities
For effectively managing the workflow and ensure continuous flow of work, it is imperative to know the bottlenecks in the construction. Subcontractor productivity is a major factor contributing to this. Therefore, the third query is identifying the low performing subcontractors in each storey.  For calculating this, the query first looks for all the activities planned for the last six weeks and then identify subcontractor responsible for the same. Then it looks at the completion status for each activity and then groups the results by subcontractor and storey. 

In [7]:
# Define a SPARQL query



query = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX cSite: <http://www.owl-ontologies.com/cSite#>
PREFIX bot: <http://www.owl-ontologies.com/bot#>
SELECT ?storey ?org ((xsd:float(COUNT(DISTINCT ?activityC))) / (xsd:float(COUNT(DISTINCT ?activity))) AS ?Completion_Rate) WHERE {
  ?weekplan cSite:CoversFrom ?date.
  ?activityC cSite:discussedIn ?weekplan;
  	cSite:StatusCompleted ?status;
    cSite:hasOrganization ?org;
    cSite:hasZone ?space1.
  ?activity cSite:discussedIn ?weekplan.
  ?activity
        cSite:StatusCompleted ?status1;
        cSite:hasOrganization ?org;
        cSite:hasZone ?space.
  ?space bot:hasStorey ?storey.
  ?space1 bot:hasStorey ?storey.
  #BIND((NOW()) - "P0Y6M28DT0M0S"^^xsd:yearMonthDuration AS ?sixweeksago)
  BIND("2022-04-01"^^xsd:date AS ?sixweeksago)
  FILTER(?date > ?sixweeksago)
  FILTER(?status = "Yes"^^xsd:string)
}
GROUP BY ?storey ?org
ORDER BY (?storey) (?Completion_Rate)

"""


results = perform_sparql_query(graph, query)

# Display results
df = display_query_results(results)
print(df)

      storey             org      Completion_Rate
0  cSite#001  cSite#CompanyL  0.45454545454545453
1  cSite#001  cSite#CompanyO    0.972972972972973
2  cSite#001  cSite#CompanyC                  1.0
3  cSite#002  cSite#CompanyL                  0.5
4  cSite#002  cSite#CompanyO   0.5974025974025974
5  cSite#003  cSite#CompanyM   0.2857142857142857
6  cSite#020  cSite#CompanyI                  1.0


This SPARQL query is designed to calculate and retrieve the completion rate of activities within different storeys and organizations, filtered by a specific time period. Here's a detailed breakdown in plain English:



1. **SELECT Clause**:
   - The query selects three variables: `?storey`, `?org`, and `?Completion_Rate`.

2. **Calculation of `?Completion_Rate`**:
   - The `?Completion_Rate` is calculated as the ratio of the number of distinct completed activities (`?activityC`) to the total number of distinct activities (`?activity`) for each combination of `?storey` and `?org`.

3. **WHERE Clause**:
   - `?weekplan cSite:CoversFrom ?date`: The `?weekplan` has a starting date of `?date`.
   - `?activityC cSite:discussedIn ?weekplan`: Completed activities (`?activityC`) are discussed in the `?weekplan`.
     - These completed activities have:
       - A status (`?status`) of "Completed".
       - An associated organization (`?org`).
       - An associated zone (`?space1`).
   - `?activity cSite:discussedIn ?weekplan`: All activities (`?activity`) are discussed in the `?weekplan`.
     - These activities have:
       - A status (`?status1`), which could be any status.
       - An associated organization (`?org`).
       - An associated zone (`?space`).
   - `?space bot:hasStorey ?storey` and `?space1 bot:hasStorey ?storey`: Both the zones `?space` and `?space1` are located on the same storey.

4. **Date Filtering**:
   - The query binds the value `2022-04-01` as `?sixweeksago`.
   - It filters the results to include only those week plans with a date (`?date`) later than `?sixweeksago`.

5. **Status Filtering**:
   - It filters the results to include only those activities where the status (`?status`) is "Yes".

6. **Grouping and Ordering**:
   - The results are grouped by `?storey` and `?org`.
   - The results are ordered by `?storey` and then by the completion rate (`?Completion_Rate`).

In summary, this query retrieves the storey and organization pairs along with the completion rate of their activities since April 1, 2022. The completion rate is the proportion of completed activities out of the total activities for each storey and organization combination.

## Query 4 : Identifying locations with least productivity

Besides determining subcontractor productivity, it is essential to consider location-based productivity as well. This is because certain locations may have inefficient logistics and working plans, which can contribute to lower overall productivity. Assessing location-based productivity provides valuable insights that cannot be solely obtained from subcontractor productivity analysis. The query algorithm begins by examining all activities conducted within the past six weeks. It then evaluates the specific spaces in which these activities took place. Finally, it checks the completion status of each activity to calculate the completion rate. By analysing this information, areas or floors with lower productivity can be pinpointed and assessed for potential issues in logistics or working plans. This enables stakeholders to address these concerns and take appropriate measures to improve overall productivity in those specific locations.

In [8]:
# Define a SPARQL query



query = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX cSite: <http://www.owl-ontologies.com/cSite#>
PREFIX bot: <http://www.owl-ontologies.com/bot#>

SELECT ?storey ((xsd:float(COUNT(DISTINCT ?activityC))) / (xsd:float(COUNT(DISTINCT ?activity))) AS ?Completion_Rate) WHERE {
  ?weekplan cSite:CoversFrom ?date.
  ?activityC cSite:discussedIn ?weekplan.
  ?activityC cSite:StatusCompleted ?status.
  ?activityC cSite:hasOrganization ?org;
    cSite:hasZone ?space1.
  ?activity cSite:discussedIn ?weekplan.
  ?activity cSite:StatusCompleted ?status1.
  ?activity cSite:hasOrganization ?org;
    cSite:hasZone ?space.
  ?space bot:hasStorey ?storey.
  ?space1 bot:hasStorey ?storey.
    #BIND((NOW()) - "P0Y6M28DT0M0S"^^xsd:yearMonthDuration AS ?sixweeksago)
  BIND("2022-04-01"^^xsd:date AS ?sixweeksago)
  FILTER(?date > ?sixweeksago)
  FILTER(?status = "Yes"^^xsd:string)
}
GROUP BY ?storey
ORDER BY (?Completion_Rate) (?storey)

"""


results = perform_sparql_query(graph, query)

# Display results
df = display_query_results(results)
print(df)

      storey     Completion_Rate
0  cSite#003  0.2857142857142857
1  cSite#002  0.5949367088607594
2  cSite#001                0.86
3  cSite#020                 1.0




### SELECT Clause
The `SELECT` clause specifies the variables to be returned in the results:

- `?storey`: A variable representing the storey of the building.
- `?Completion_Rate`: A variable representing the completion rate of activities on that storey.

### WHERE Clause
The `WHERE` clause contains the conditions that must be satisfied for the data to be included in the results:

1. **Date Filtering:**
   - `?weekplan cSite:CoversFrom ?date`: Each `weekplan` covers a certain date.
   - `BIND("2022-04-01"^^xsd:date AS ?sixweeksago)`: The variable `?sixweeksago` is set to the date April 1, 2022.
   - `FILTER(?date > ?sixweeksago)`: Only include `weekplans` with dates after April 1, 2022.

2. **Completed Activities:**
   - `?activityC cSite:discussedIn ?weekplan`: Activities discussed in the weekplan.
   - `?activityC cSite:StatusCompleted ?status`: The status of these activities is marked as completed.
   - `?activityC cSite:hasOrganization ?org`: The activities are associated with an organization.
   - `?activityC cSite:hasZone ?space1`: The activities are associated with a zone.

3. **All Activities:**
   - `?activity cSite:discussedIn ?weekplan`: Activities discussed in the weekplan.
   - `?activity cSite:StatusCompleted ?status1`: The status of these activities.
   - `?activity cSite:hasOrganization ?org`: The activities are associated with an organization.
   - `?activity cSite:hasZone ?space`: The activities are associated with a zone.

4. **Storey Mapping:**
   - `?space bot:hasStorey ?storey`: The zone is mapped to a storey of the building.
   - `?space1 bot:hasStorey ?storey`: Another zone is mapped to the same storey.

5. **Filter for Completed Status:**
   - `FILTER(?status = "Yes"^^xsd:string)`: Only include activities that have been completed (`StatusCompleted` equals "Yes").

### GROUP BY and ORDER BY Clauses
- `GROUP BY ?storey`: The results are grouped by the storey.
- The completion rate (`?Completion_Rate`) is calculated by dividing the count of distinct completed activities by the count of all distinct activities.
- The results are ordered first by the completion rate and then by the storey.

### Final Output
The query will output the storey of the building along with the completion rate of activities on that storey, considering only activities discussed in weekplans after April 1, 2022, and that have been marked as completed. The results will be sorted by the completion rate and then by the storey.

## Query 5 : Identifying subcontractor completion rates

In addition to obtaining subcontractor productivities per floor, it is equally crucial to determine the overall completion rate of subcontractors. This information plays a vital role in identifying underperforming subcontractors and taking necessary corrective actions. Listing 5 presents a query specifically designed to calculate the completion rate of various subcontractors on a weekly basis. The query algorithm retrieves all activities performed within the last six weeks and identifies the responsible subcontractor for each activity. Subsequently, it checks whether each activity has been completed and aggregates the data on a weekly basis for each subcontractor. TThis data aids in the identification of subcontractors with lower performance and assists in implementing appropriate measures to rectify any issues.

In [9]:
# Define a SPARQL query



query = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX cSite: <http://www.owl-ontologies.com/cSite#>
SELECT ?weekplan ?org (COUNT(DISTINCT ?activityC) AS ?N_completed_activities) (COUNT(DISTINCT ?activity) AS ?N_activities) ((xsd:float(COUNT(DISTINCT ?activityC))) / (xsd:float(COUNT(DISTINCT ?activity))) AS ?Completion_Rate) WHERE {
  ?weekplan cSite:CoversFrom ?date.
  ?activityC cSite:discussedIn ?weekplan.
  ?activityC cSite:StatusCompleted ?status.
  ?activityC cSite:hasOrganization ?org.
  ?activity cSite:discussedIn ?weekplan.
  ?activity cSite:StatusCompleted ?status1.
  ?activity cSite:hasOrganization ?org.
   #BIND((NOW()) - "P0Y6M28DT0M0S"^^xsd:yearMonthDuration AS ?sixweeksago)
  BIND("2022-04-01"^^xsd:date AS ?sixweeksago)
  FILTER(?date > ?sixweeksago)
  FILTER(?status = "Yes"^^xsd:string)
}
GROUP BY ?weekplan ?org
ORDER BY (?weekplan) DESC (?Completion_Rate)


"""


results = perform_sparql_query(graph, query)

# Display results
df = display_query_results(results)
print(df)

                                    weekplan             org  \
0   cSite#THISWEEK-WeeklyPMUpdate220408.xlsx  cSite#CompanyC   
1   cSite#THISWEEK-WeeklyPMUpdate220408.xlsx  cSite#CompanyO   
2   cSite#THISWEEK-WeeklyPMUpdate220415.xlsx  cSite#CompanyC   
3   cSite#THISWEEK-WeeklyPMUpdate220415.xlsx  cSite#CompanyO   
4   cSite#THISWEEK-WeeklyPMUpdate220415.xlsx  cSite#CompanyL   
5   cSite#THISWEEK-WeeklyPMUpdate220422.xlsx  cSite#CompanyO   
6   cSite#THISWEEK-WeeklyPMUpdate220429.xlsx  cSite#CompanyI   
7   cSite#THISWEEK-WeeklyPMUpdate220429.xlsx  cSite#CompanyO   
8   cSite#THISWEEK-WeeklyPMUpdate220506.xlsx  cSite#CompanyO   
9   cSite#THISWEEK-WeeklyPMUpdate220506.xlsx  cSite#CompanyL   
10  cSite#THISWEEK-WeeklyPMUpdate220506.xlsx  cSite#CompanyM   
11  cSite#THISWEEK-WeeklyPMUpdate220527.xlsx  cSite#CompanyO   

   N_completed_activities N_activities     Completion_Rate  
0                       1            1                 1.0  
1                       7            7       

This SPARQL query retrieves data about activities and their completion rates within specific week plans, organized by organizations. Here's a breakdown of what the query does:

1. **Select Variables:**
   - `?weekplan`: The specific week plan.
   - `?org`: The organization associated with the activities.
   - `?N_completed_activities`: The number of completed activities.
   - `?N_activities`: The total number of activities.
   - `?Completion_Rate`: The rate of completed activities to total activities.

2. **Conditions and Filters:**
   - The query checks for activities discussed in the context of a week plan.
   - It specifically looks for activities marked as completed (`?status = "Yes"`).
   - The activities must belong to an organization (`?org`).
   - The week plan must have a date (`?date`) greater than six weeks ago (specified as `2022-04-01`).

3. **Grouping and Counting:**
   - The query groups the results by week plan and organization.
   - It counts the distinct number of completed activities and total activities within these groups.

4. **Calculating Completion Rate:**
   - The completion rate is calculated as the ratio of completed activities to the total activities.

5. **Ordering the Results:**
   - The results are first ordered by week plan in descending order.
   - Then, they are ordered by completion rate and storey (though the `?storey` variable is not defined in the provided query, so this part seems redundant or may be a mistake).

### Detailed Explanation:
1. **Selection Clause (`SELECT`):**
   - `?weekplan`: Represents each week plan.
   - `?org`: Represents each organization.
   - `(COUNT(DISTINCT ?activityC) AS ?N_completed_activities)`: Counts the distinct completed activities.
   - `(COUNT(DISTINCT ?activity) AS ?N_activities)`: Counts the distinct total activities.
   - `((xsd:float(COUNT(DISTINCT ?activityC))) / (xsd:float(COUNT(DISTINCT ?activity))) AS ?Completion_Rate)`: Calculates the completion rate as a float.

2. **Where Clause (`WHERE`):**
   - `?weekplan cSite:CoversFrom ?date`: Links the week plan to a date.
   - `?activityC cSite:discussedIn ?weekplan`: Links completed activities to the week plan.
   - `?activityC cSite:StatusCompleted ?status`: Filters for completed activities.
   - `?activityC cSite:hasOrganization ?org`: Associates activities with an organization.
   - `?activity cSite:discussedIn ?weekplan`: Links all activities to the week plan.
   - `?activity cSite:StatusCompleted ?status1`: Associates all activities with a status.
   - `?activity cSite:hasOrganization ?org`: Associates all activities with an organization.
   - `BIND("2022-04-01"^^xsd:date AS ?sixweeksago)`: Binds a specific date to the variable `?sixweeksago`.
   - `FILTER(?date > ?sixweeksago)`: Filters week plans to only include those after `2022-04-01`.
   - `FILTER(?status = "Yes"^^xsd:string)`: Filters to include only completed activities.

3. **Grouping (`GROUP BY`):**
   - Groups results by week plan and organization.

4. **Ordering (`ORDER BY`):**
   - Orders the results first by week plan in descending order.
   - Orders next by completion rate.
   - The final ordering by `?storey` is likely incorrect or unnecessary given the provided context.

### Summary:
The query fetches and calculates the number of total and completed activities for each week plan and organization, then computes the completion rate. It filters the results to only include week plans from the past six weeks and sorts the results by week plan and completion rate.

## Query 6 Identifying reasons for non-completions

Although identifying the completion rate can help identify the subcontractor productivity. For making the best rectifying action, it is also necessary to identify the reasons for non-completion. The query first identifies the activities which are not completed in the last 6 weeks and associates the organisation with these activities and the calculates which has the highest no of non-completions. Then it looks for the reasons for the non-completion and counts them. 

In [10]:
# Define a SPARQL query



query = """
PREFIX cSite: <http://www.owl-ontologies.com/cSite#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?org ?reason (COUNT(?reason) AS ?reasoncount) WHERE {
  ?activity cSite:hasOrganization ?org.
  ?activity cSite:discussedIn ?weekplan.
  
  ?activity cSite:ReasonForDelay ?reason.
  {
    SELECT ?org (COUNT(?activity) AS ?count) WHERE {
      ?weekplan cSite:CoversFrom ?date.
      ?activity cSite:StatusCompleted ?status.
      ?activity cSite:hasOrganization ?org.
      #BIND((NOW()) - "P0Y6M28DT0M0S"^^xsd:yearMonthDuration AS ?sixweeksago)
  	  BIND("2022-04-01"^^xsd:date AS ?sixweeksago)
      FILTER(?date > ?sixweeksago)
      FILTER(?status = "No"^^xsd:string)
    }
    GROUP BY ?org
    ORDER BY DESC (?count)
    LIMIT 10
  }
}
GROUP BY ?org ?reason
ORDER BY DESC (?reasoncount)
LIMIT 10


"""


results = perform_sparql_query(graph, query)

# Display results
df = display_query_results(results)
print(df)

              org                                       reason reasoncount
0  cSite#CompanyM                           Activity Durations          64
1  cSite#CompanyM     Coordination of Trades at the Work Front          45
2  cSite#CompanyL                 Labour Resource Availability          30
3  cSite#CompanyM                         Access and Logistics          22
4  cSite#CompanyO                           Activity Durations          18
5  cSite#CompanyK                 Labour Resource Availability          18
6  cSite#CompanyL                           Activity Durations          14
7  cSite#CompanyO  Subcontractor & Supplier design Information          12
8  cSite#CompanyL                             Outanding Design          11
9  cSite#CompanyM                           Construction Logic          10


This SPARQL query involves two parts: an inner subquery and an outer query. Let's break down what each part does in plain English.

### Inner Subquery:
The inner subquery selects organizations with the highest number of activities not completed in the last six weeks (up to April 1, 2022). Here's a step-by-step explanation:

1. **Identifying Relevant Activities:**
   - It looks for activities (`?activity`) associated with organizations (`?org`).
   - Each activity should be covered in a weekly plan (`?weekplan`).

2. **Time and Completion Status:**
   - The weekly plan (`?weekplan`) must cover a date (`?date`) that is after April 1, 2022.
   - The activity must not be completed (`?status = "No"`).

3. **Counting Activities:**
   - It counts the number of such activities for each organization (`COUNT(?activity) AS ?count`).

4. **Grouping and Sorting:**
   - Groups the results by organization (`GROUP BY ?org`).
   - Orders the organizations by the number of incomplete activities in descending order (`ORDER BY DESC(?count)`).
   - Limits the results to the top 10 organizations with the most incomplete activities (`LIMIT 10`).

### Outer Query:
The outer query uses the results from the inner subquery to find the reasons for delays associated with activities from the top 10 organizations. Here's a step-by-step explanation:

1. **Selecting Organizations and Reasons:**
   - For each activity (`?activity`), it identifies the associated organization (`?org`) and the reason for delay (`?reason`).
   - It ensures the activity is part of the same top 10 organizations identified in the inner subquery.

2. **Counting Reasons:**
   - It counts the number of occurrences of each reason for delay for each organization (`COUNT(?reason) AS ?reasoncount`).

3. **Grouping and Sorting:**
   - Groups the results by organization and reason (`GROUP BY ?org ?reason`).
   - Orders the reasons by the count of occurrences in descending order (`ORDER BY DESC (?reasoncount)`).
   - Limits the results to the top 10 reasons (`LIMIT 10`).

### Summary:
In summary, this SPARQL query identifies the top 10 organizations with the most activities not completed in the last six weeks (up to April 1, 2022) and then finds the most common reasons for delays in these activities. The results show the organizations, the reasons for delays, and the count of each reason, sorted by the frequency of the reasons.

## Query 7 Identifying activity that has been replanned the most, the reason and who is most affected

Frequent replanning of certain activities is a significant aspect that warrants investigation. Understanding the reasons behind these replans, as well as identifying the organizations involved and affected, is crucial. The query examines the number of times each activity has been replanned, investigates the reasons behind the replanning, identifies the responsible organization for non-completion, and determines the organization most impacted by these replans. The resulting output will provide valuable insights into the replanning dynamics and their organizational implications.

In [11]:
# Define a SPARQL query



query = """
PREFIX cSite: <http://www.owl-ontologies.com/cSite#>
PREFIX bot: <http://www.owl-ontologies.com/bot#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?storey ?org ?activity ?reason ?resporg WHERE {
  ?weekplan cSite:CoversFrom ?date.
  ?activity cSite:discussedIn ?weekplan;
    cSite:hasOrganization ?org;
    cSite:hasZone ?space.
  ?space bot:hasStorey ?storey.
  ?activity cSite:PlannedVersion ?version;
    cSite:ReasonForDelay ?reason;
    cSite:responsibleForNonCompletion ?resporg.
  FILTER((xsd:integer(?version)) > 1 )
  FILTER(?date > ?sixweeksago)
  {
    SELECT ?storey ?org (COUNT(?activity) AS ?n_orgreplanned) ?sixweeksago WHERE {
      ?weekplan cSite:CoversFrom ?date.
      ?activity cSite:discussedIn ?weekplan.
      ?activity cSite:hasOrganization ?org;
        cSite:hasZone ?space.
      ?space bot:hasStorey ?storey.
      ?activityversion cSite:PlannedVersion ?version.
      FILTER(?date > ?sixweeksago)
      FILTER((xsd:integer(?version)) > 1 )
      {
        SELECT ?storey (COUNT(DISTINCT ?activityversion) AS ?n_replanned) ?sixweeksago WHERE {
          ?weekplan cSite:CoversFrom ?date.
          ?activity cSite:discussedIn ?weekplan.
          ?activity  cSite:hasZone ?space.
          ?space bot:hasStorey ?storey.
          ?activityversion cSite:PlannedVersion ?version.
          #BIND((NOW()) - "P0Y6M28DT0M0S"^^xsd:yearMonthDuration AS ?sixweeksago)
  	  	  BIND("2022-04-01"^^xsd:date AS ?sixweeksago)
          FILTER(?date > ?sixweeksago)
          FILTER((xsd:integer(?version)) > 1 )
        }
        GROUP BY ?storey ?sixweeksago
        ORDER BY DESC (?n_replanned)
        LIMIT 4
      }
    }
    GROUP BY ?storey ?sixweeksago ?org
    LIMIT 4
  }
}
GROUP BY ?storey ?org ?activity ?reason ?resporg
ORDER BY (?activity)


"""


results = perform_sparql_query(graph, query)

# Display results
df = display_query_results(results)
print(df)

       storey             org                 activity  \
0   cSite#020  cSite#CompanyI  cSite#3200-QHP-020-0202   
1   cSite#020  cSite#CompanyI  cSite#3200-QHP-020-0202   
2   cSite#020  cSite#CompanyI  cSite#3200-QHP-020-0316   
3   cSite#020  cSite#CompanyI  cSite#3200-QHP-020-0316   
4   cSite#002  cSite#CompanyL  cSite#4255-QSO-002-0020   
5   cSite#002  cSite#CompanyL  cSite#4255-QSO-002-0020   
6   cSite#002  cSite#CompanyL  cSite#4255-QSO-002-0022   
7   cSite#002  cSite#CompanyL  cSite#4255-QSO-002-0022   
8   cSite#002  cSite#CompanyL  cSite#4255-QSO-002-0112   
9   cSite#002  cSite#CompanyL  cSite#4255-QSO-002-0112   
10  cSite#002  cSite#CompanyL  cSite#4255-QSO-002-0112   
11  cSite#002  cSite#CompanyL  cSite#4255-QSO-002-0112   
12  cSite#002  cSite#CompanyL  cSite#4255-QSO-002-0113   
13  cSite#002  cSite#CompanyL  cSite#4255-QSO-002-0113   
14  cSite#002  cSite#CompanyL  cSite#4255-QSO-002-0113   
15  cSite#002  cSite#CompanyL  cSite#4255-QSO-002-0113   
16  cSite#002 



### Main Query
The main query aims to retrieve detailed information about activities in a construction site, specifically focusing on those that have been replanned. It gathers data on the storey, organization, activity, reason for delay, and responsible organization for non-completion of the activity.

#### Outer SELECT
```sparql
SELECT ?storey ?org ?activity ?reason ?resporg WHERE {
  ...
  GROUP BY ?storey ?org ?activity ?reason ?resporg
  ORDER BY (?activity)
}
```
This part of the query selects the storey, organization, activity, reason for delay, and responsible organization for non-completion, grouping the results and ordering them by activity.

### First Subquery
The first subquery within the main query aims to filter activities based on their version (greater than 1) and date (within the last six weeks).

```sparql
?weekplan cSite:CoversFrom ?date.
?activity cSite:discussedIn ?weekplan;
  cSite:hasOrganization ?org;
  cSite:hasZone ?space.
?space bot:hasStorey ?storey.
?activity cSite:PlannedVersion ?version;
  cSite:ReasonForDelay ?reason;
  cSite:responsibleForNonCompletion ?resporg.
FILTER((xsd:integer(?version)) > 1 )
FILTER(?date > ?sixweeksago)
```

This part finds activities that:
- Have a version number greater than 1.
- Were discussed in a week plan that covers a date within the last six weeks.
- Are linked to an organization and a specific zone within a storey.
- Have a reason for delay and a responsible organization for non-completion.

### Second Subquery
The second subquery calculates the count of replanned activities by organization and storey, filtering based on the same date and version criteria.

```sparql
{
  SELECT ?storey ?org (COUNT(?activity) AS ?n_orgreplanned) ?sixweeksago WHERE {
    ...
    GROUP BY ?storey ?sixweeksago ?org
    LIMIT 4
  }
}
```

This part:
- Selects the storey, organization, and counts the number of replanned activities.
- Groups results by storey, the six-week period, and organization.
- Limits the results to the top 4.

### Third Subquery
The third subquery is nested inside the second subquery and aims to find the top 4 storeys with the highest number of replanned activities.

```sparql
{
  SELECT ?storey (COUNT(DISTINCT ?activityversion) AS ?n_replanned) ?sixweeksago WHERE {
    ...
    GROUP BY ?storey ?sixweeksago
    ORDER BY DESC (?n_replanned)
    LIMIT 4
  }
}
```

This part:
- Selects the storey and counts the number of distinct replanned activities.
- Filters by date (within the last six weeks) and version (greater than 1).
- Groups by storey and the six-week period.
- Orders results by the number of replanned activities in descending order.
- Limits the results to the top 4 storeys.

### Date Filtering
The BIND statement sets a static date to represent six weeks ago:

```sparql
BIND("2022-04-01"^^xsd:date AS ?sixweeksago)
```

### Final Output
The final output will provide a list of activities that meet the criteria, including:
- Storey
- Organization responsible for the activity
- Activity itself
- Reason for the delay
- Organization responsible for non-completion

These results are grouped and ordered to present a clear view of the most replanned activities within the specified period and their associated details.